In [1]:
#Make separate data files for each lineage

In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from augur.utils import json_to_tree
import json

In [3]:
#need to look at tree json file to find the branch name for the common ancestor of the different lineages
#format of dict {lineage:common_ancestor}


genotype_ancestors = {'A': 'NODE_0000035', 'B':'NODE_0000001'}

In [4]:
for lineage, ca in genotype_ancestors.items():
    #read in tree
    with open(f'../auspice/oc43_all_genome.json', 'r') as f:
        tree_json = json.load(f)

    #Put tree in Bio.Phylo format
    tree = json_to_tree(tree_json)
    
    #store all tip names for each subgenotype
    tips_in_this_lineage = []

    
    #find tips descending from common ancestor branch of the lineages
    for node in tree.find_clades(terminal=True):
        parents = [x.name for x in tree.get_path(node)]
        if ca in parents:
            tips_in_this_lineage.append(node.name)
                        
    seq_records = []

    for record in SeqIO.parse(open(f"oc43_all_genome.fasta","r"), "fasta"):
        accession = record.description.split('|')[0]
        #check for the tips that cluster with this genotype
        if accession in tips_in_this_lineage:
            seq_records.append(SeqRecord(record.seq, id=record.description, description=''))

        
    SeqIO.write(seq_records, f'oc43_{lineage}_genome.fasta', "fasta")